# Data Manipulation with Expressions: Creating and Modifying Columns

- Add, transform or create new columns using `df.with_columns([...])`
- Conditional transfomations with `pl.when().then().otherwise()`
- Common transformations with expression API: arithmetic, string, datetime
- Bonus: programmatic expressions 

Goal: Appreciate Polars' parallelized transformation expressions inside `.with_columns([])`


## Expressions in Context: `select` vs `with_columns`


In [ ]:
import polars as pl

df = pl.DataFrame(
    {
        "coffee": ["Espresso", "Latte", "Mocha"],
        "price": [2.5, 3.0, 3.5],
        "sold": [10, 8, 6],
    }
)

df

In [ ]:
# expressions:
# - declare recipe
# - no data involved so far
revenue_expression = (pl.col("price") * pl.col("sold")).alias("revenue")
revenue_expression

In [ ]:
# contexts execute expressions
# `.select()` creates a new DataFrame with selected columns
df.select(revenue_expression)

In [ ]:
# add new column with `.with_columns(...)` context
expanded_df = df.with_columns(revenue_expression)
expanded_df

In [ ]:
# no-inplace mutation of original DataFrame
df

In [ ]:
# assign multiple columns at once
# fully parallelized!
expanded_df_2 = df.with_columns(
    [
        revenue_expression,
        pl.lit("Berlin").alias("city"),
    ]
)
expanded_df_2

In [ ]:
# mind using `.alias()` to name a new column

# ❌ shadows "price"
df_bad = df.with_columns([pl.col("price") * pl.col("sold")])

# ✅ `.alias()` prevents shadowing
df_good = df.with_columns([(pl.col("price") * pl.col("sold")).alias("revenue")])

df_bad, df_good

## ☕ Load a Realistic Dataset (Coffee Sales)


In [ ]:
# load dataset
import polars as pl
from pathlib import Path

if Path("coffee_sales.parquet").exists():
    coffee_sales = pl.read_parquet("coffee_sales.parquet")
else:
    # minimal inline dataset so the lesson is self-contained
    coffee_sales = pl.DataFrame(
        {
            # stock keeping unit
            "sku": [
                "ESP-001",  # Espresso
                "ESP-002",  # Double Espresso
                "LAT-001",  # Latte
                "WAT-001",  # Water Still
                "WAT-002",  # Water Sparkling
                "TEA-001",  # Green Tea
                "SPC-001",  # Matcha Latte
            ],
            "drink": [
                "Espresso",
                "Double Espresso",
                "Latte",
                "Water Still",
                "Water Sparkling",
                "Green Tea",
                "Matcha Latte",
            ],
            "price": [3.0, 4.0, 4.0, 2.0, 2.2, 2.5, 4.8],
            "quantity": [1, 2, 1, 3, 1, 3, 1],
            "timestamp": pl.datetime_range(
                start=pl.datetime(2025, 9, 1),
                end=pl.datetime(2025, 10, 7),
                interval="1d",
                eager=True,
            ),
            "city": [
                "Berlin",
                "Austin",
                "New York",
                "Los Angeles",
                "Austin",
                "San Francisco",
                "San Francisco",
            ],
        }
    )

print(f"Length of full DataFrame: {len(coffee_sales)}")
coffee_sales.head(8)


## Conditional Logic with `when` / `then` / `otherwise`


In [ ]:
# City-based discount rules
# Austin 10%, San Francisco 8%, Berlin 5%, others 0%
discount_expr = (
    pl.when(pl.col("city") == "Austin")
    .then(0.10)
    .when(pl.col("city") == "San Francisco")
    .then(0.08)
    .when(pl.col("city") == "Berlin")
    .then(0.05)
    .otherwise(0.00)
    .alias("discount_pct")
)

coffee_sales_discounts = coffee_sales.with_columns([discount_expr])

coffee_sales_discounts.tail(8)


## Arithmetic Columns: Revenue, Net Price, Chaining

In [ ]:
discounted = coffee_sales_discounts.with_columns(
    [
        # comparison expression creates boolean mask
        (pl.col("discount_pct") > 0).alias("discount_active"),
        # Arithmetic demos: *, -, /
        (pl.col("price") * pl.col("quantity")).alias("gross_revenue"),
        (pl.col("price") * (1 - pl.col("discount_pct"))).alias("net_price"),
    ]
)

discounted.select(
    "discount_active",
    "city",
    "quantity",
    "price",
    "gross_revenue",
    "discount_pct",
    "net_price"
)

In [ ]:
# chaining makes intermediate column results available for
# the next context
discounted.with_columns(
    [
        # discount per item
        (pl.col("price") - pl.col("net_price")).alias("discount_amount"),
        # 
        (pl.col("net_price") * pl.col("quantity")).alias("net_revenue"),
    ]
).with_columns(
    [
        # Simple per-unit division demo
        (pl.col("net_revenue") / pl.col("quantity")).alias("avg_net_price_per_unit"),
    ]
).select(
    "quantity",
    "price",
    "discount_active",
    "discount_amount",
    "net_price",
    "gross_revenue",
    "net_revenue",
    "avg_net_price_per_unit",
)


In [ ]:
# Finally remember you can also run all manipulations in parallel
single_chain = (
    coffee_sales.with_columns(
        [
            discount_expr,
        ]
    )
    .with_columns(
        [
            (pl.col("discount_pct") > 0).alias("discount_active"),
            # Arithmetic demos: *, -, /
            (pl.col("price") * pl.col("quantity")).alias("gross_revenue"),
            (pl.col("price") * (1 - pl.col("discount_pct"))).alias("net_price"),
        ]
    )
    .with_columns(
        [
            (pl.col("price") - pl.col("net_price")).alias("discount_amount"),
            (pl.col("net_price") * pl.col("quantity")).alias("net_revenue"),
        ]
    )
    .with_columns(
        [
            # Simple per-unit division demo
            (pl.col("net_revenue") / pl.col("quantity")).alias(
                "avg_net_price_per_unit"
            ),
        ]
    )
)

single_chain.select(
    "quantity",
    "price",
    "discount_active",
    "discount_amount",
    "net_price",
    "gross_revenue",
    "net_revenue",
    "avg_net_price_per_unit",
)

## String Manipulation with `.str` (patterns, regex, case)



Expression API availalable under `expr.str.` attribute.

Full overview: http://docs.pola.rs/api/python/stable/reference/expressions/string.html


In [ ]:
# check string pattern
selected = coffee_sales.select(
    pl.col("drink"),
    pl.col("sku"),
    pl.col("sku").str.starts_with("WAT").alias("is_water"),
    pl.col("sku").str.ends_with("002").alias("second_of_class"),
    pl.col("sku").str.contains("001").alias("first_item_of_class"),
    # contains with . as placeholder
    pl.col("sku").str.contains("AT...1").alias("first_water_or_latte"),
)
selected.tail(10)

In [ ]:
# regex for extraction, pattern matching or replacement
result = coffee_sales.select(
    pl.col("sku").str.extract(r"(\d+)").alias("class_item"),
    pl.col("sku").str.replace("SPC", "SPECIAL").alias("special"),
)
result

In [ ]:
# string manipulations
lowercase_drinks = pl.col("drink").str.to_lowercase().alias("drink_lowercase")
lowercase_drinks

In [ ]:
coffee_sales.with_columns(
    [
        lowercase_drinks,
        pl.col("drink").str.to_uppercase().alias("drink_uppercase"),
    ]
).select(pl.col("^drink.*$"))

In [ ]:
# string cropping
coffee_sales.select(
    pl.col("sku").str.head(3).alias("sku_classes_v1"),
    pl.col("sku").str.split("-").alias("splitted"),
    pl.col("sku").str.split("-").list.get(0).alias("sku_classes_v2"),
)

In [ ]:
# categorize drinks in higher level categories
assign_category = (
    pl.when(pl.col("sku").str.starts_with("ESP") | pl.col("sku").str.starts_with("LAT"))
    .then(pl.lit("Coffee"))
    .when(pl.col("sku").str.starts_with("WAT") | pl.col("sku").str.starts_with("LEM"))
    .then(pl.lit("Bottles"))
    .when(pl.col("sku").str.starts_with("SPC"))
    .then(pl.lit("Special"))
    .otherwise(pl.lit("Other"))
    .alias("category")
)
coffee_sales.with_columns(assign_category).select("sku", "drink", "category")

## Temporal Features with `.dt` (weekday, business day)



Expression API availalable under `expr.dt` attribute.

Full overview: https://docs.pola.rs/api/python/stable/reference/expressions/temporal.html


In [ ]:
day_of_week = pl.col("timestamp").dt.strftime("%A").alias("day_of_week_string")

coffee_sales.select(
    pl.col('timestamp'),
    day_of_week,
    pl.col("timestamp").dt.strftime("%Y-%m-%d").alias('date_string')
)

In [ ]:
coffee_sales.select(
    pl.col("timestamp"),
    day_of_week,
    pl.col("timestamp").dt.weekday().alias("day_of_week"),
    pl.col("timestamp").dt.day().alias("day_of_month"),
    pl.col("timestamp").dt.ordinal_day().alias("day_of_year"),
).sort("timestamp")


In [ ]:
# extracting business days
# default:  Monday - Friday without holidays
# custom: weekday mask and/or integrate holidays

coffee_sales.select(

    # assign boolean mask
    pl.col("timestamp").dt.is_business_day().alias("is_business_day"),
    # negate to get weekend
    ~pl.col("timestamp").dt.is_business_day().alias("is_weekend"),

    # create category
    pl.when(pl.col("timestamp").dt.is_business_day())
    .then(pl.lit("Weekday"))
    .otherwise(pl.lit("Weekend"))
    .alias("day_type"),

).tail(10)

## 🎁 Bonus Recap and Programmatic Expressions


In [ ]:
coffee_sales.with_columns(
    # add conditional category
    assign_category

).filter(
    # use derived category
    pl.col("category").is_in(["Coffee", "Special"]),

    # namespaces work in filter as well
    pl.col("timestamp").dt.is_business_day(),

).select("category","drink", day_of_week).head(10)


In [ ]:
%%time
# programmatic expression generation

# generate 3 scaled price columns without loops
scales = [1.1, 1.2, 1.3, 1.4, 1.5]
exprs = [(pl.col("price") * s).alias(f"price_x{s}") for s in scales]
parallel = coffee_sales.with_columns(exprs)
parallel.select("drink", pl.col("^price.*$")).head()

In [ ]:
%%time
# programmatic expression generation
sequential = coffee_sales
for expr in exprs:
    sequential = sequential.with_columns(expr)
sequential.select("drink", pl.col("^price.*$")).head()

## Wrap-Up


- Use `df.with_columns([...])` to **add or modify** columns.
- Each argument inside is a **Polars expression**, like `pl.col("price") * 2`.
- Use `.alias("new_name")` to name the result.
- Combine multiple expressions in one call — Polars runs them **in parallel**.
- Use `pl.when(...).then(...).otherwise(...)` for **conditional logic** (instead of Python `if/else`).
- String and datetime columns have their own namespaces: `.str` and `.dt`.
- Polars never modifies data in place — always assign the result to a new variable.


In [ ]:
# ⚙️ Polars – Transformations & New Columns

# ADD OR MODIFY COLUMNS
# MULTIPLE EXPRESSIONS (computed in parallel)
coffee_sales.with_columns(
    [
        (pl.col("price") * pl.col("quantity")).alias("revenue"),
        (pl.col("price") * 1.1).alias("price_usd"),  # update or replace
    ]
)

# CONDITIONAL LOGIC (vectorized if-else)
coffee_sales.with_columns(
    pl.when(pl.col("city") == "Austin")
    .then(0.10)
    .when(pl.col("city") == "Berlin")
    .then(0.05)
    .otherwise(0.00)
    .alias("discount_pct")
)

# STRING OPERATIONS with `expr.str.` namespace
coffee_sales.with_columns(
    [
        pl.col("drink").str.to_lowercase().alias("drink_lower"),
        pl.col("sku").str.starts_with("ESP").alias("is_espresso"),
    ]
)

# DATE/TIME OPERATIONS with `expr.dt.` namespace
coffee_sales.with_columns(
    [
        pl.col("timestamp").dt.year().alias("year"),
        pl.col("timestamp").dt.strftime("%b").alias("month_abbr"),
    ]
)

# UPDATE EXISTING COLUMNS (by aliasing same name)
coffee_sales = coffee_sales.with_columns((pl.col("price") * 0.9).alias("price"))

print("Done - Happy Coding!")